In [28]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from textblob import TextBlob




In [10]:
business_json_path = '/Users/amelialei/yelp_dataset/yelp_academic_dataset_business.json'
bus_df = pd.read_json(business_json_path, lines = True)


In [11]:
bus_df = bus_df[bus_df['is_open'] == 1]


In [12]:
drop_cols = ['is_open', 'review_count']
bus_df = bus_df.drop(drop_cols, axis =1)


In [13]:
bus_df = bus_df[bus_df['categories'].str.contains('Bubble Tea|Coffee and Tea', case = False, na = False)]

In [14]:
review_json_path = '/Users/amelialei/yelp_dataset/yelp_academic_dataset_review.json'

In [15]:
size = 1000000
review = pd.read_json(review_json_path, lines=True, dtype = {'review_id': str, 'user_id': str, 'business_id': str, 'stars': int, 'date': str, 'text': str, 'useful': int, 'funny': int, 'cool': int}, chunksize = size)

In [16]:
chunk_list = []
for review_chunk in review:
    review_chunk = review_chunk.drop(['review_id', 'useful', 'funny', 'cool'], axis=1)
    review_chunk = review_chunk.rename(columns={'stars': 'review_stars'})
    chunk_merged = pd.merge(bus_df, review_chunk, on='business_id', how='inner')
    chunk_list.append(chunk_merged)
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [41]:
df['text'] = (df['text']
              .str.lower()
              .replace(r'\band\b|\bor\b|\bthe\b|\bis\b|\bto\b', '', regex=True)
              .replace(r'[^A-Za-z\s]', '', regex=True))

ps = PorterStemmer()
def tokenize_and_stem(review):
    tokens = word_tokenize(review)
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

cleaned_reviews = df['text'].apply(tokenize_and_stem)
df['text'] = cleaned_reviews
df['sentiment'] = df['text'].apply(lambda review: TextBlob(review).sentiment.polarity)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'attributes', 'categories', 'hours',
       'user_id', 'review_stars', 'text', 'date', 'sentiment'],
      dtype='object')

,name,sentiment
0,1 2 Tea,0.294040
1,"1, 2, Tea",0.266467
2,101 Taiwanese Cuisine,0.221637
3,32 Below,0.302459
4,35 Below,0.273622
...,...,...
295,Yummy House,-0.030602
296,Yummy Pho,0.293740
297,Zen Q,0.229454
298,ZenfiniTea,0.192538


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories,hours,user_id,review_stars,text,date,sentiment
8428,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",5_5iY-2eU4LMJRaMK2iE4Q,5,i love thi place just visit for first time tod...,2018-05-28 21:07:56,0.116667
8429,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",lXhTU2K2Hqh_aCjtdTNGiw,3,thi place mediocr noth stood out for us we dec...,2018-07-08 16:09:41,0.311688
8430,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",vvTjlky6KcICeTJBtcaoIg,1,thi probabl worst boba spot ive found not onli...,2018-07-09 18:19:37,-0.113384
8431,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",ii0GVHr-fw1qiz4feXP49A,4,im from sf i happen be in reno on labor day so...,2018-09-04 02:56:21,0.392500
8432,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",P8BmDXrJq3a1-HMbsCDoOQ,5,our vacat wouldnt be complet without thai milk...,2018-07-10 05:10:15,0.150000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9674,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",CsVpAEIpymc2grb9i521uA,5,boba may be new reno but not new me in la ther...,2020-01-10 07:19:34,0.172944
9675,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",ybN5igJTmnkFHXU4qseiDA,4,i have a new quest in town tri all bubbl tea i...,2019-04-11 01:05:54,0.207708
9676,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",ooPAEYfz67q-rKKZbqHuow,3,custom servic i experienc here wa amaz i wa ha...,2018-08-16 17:54:54,0.217593
9677,qKvQzhukpomdY3ITSSJ5eg,"1, 2, Tea",655 Booth St,Reno,NV,89509,39.515599,-119.827502,4.0,"{'DogsAllowed': 'False', 'BusinessParking': '{...","Bubble Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '12:0-21:0', ...",lGHqIJSoE8sE8oyq1SD7Lg,5,came here on a midday afternoon paul had a ver...,2020-01-27 21:14:39,0.325000
